* https://www.sassymamasg.com/learn-special-needs-school-singapore-preschools-early-intervention-therapy-centres-international-schools/
* https://www.enablingguide.sg/im-looking-for-disability-support/education
* https://www.moe.gov.sg/special-educational-needs/sped-schools

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ OTHERS ###################
import time
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Define Functions

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

def get_address(name):
    try:
        name="+".join(name.split()).lower()
        link = 'https://www.google.com/search?q={}+singapore+address&rlz=1C1CHBF_enIN956IN956&biw=1536&bih=722&sxsrf=AOaemvLEd4fnbeucl84jIZUE-ytjq3C4CQ%3A1631607088194&ei=MFlAYf-fC9fcz7sP9OGQkAQ&oq=abc+center+singapore+singapore+address&gs_lcp=Cgdnd3Mtd2l6EAMyBwghEAoQoAE6BwgAEEcQsAM6BAgjECc6BQgAEIAEOgYIABAWEB46BQghEKABOgQIIRAVSgQIQRgAUPi4Blj89AZg3fUGaAJwAngAgAHLAYgBtReSAQYwLjE4LjGYAQCgAQHIAQjAAQE&sclient=gws-wiz&ved=0ahUKEwj_952Hgv7yAhVX7nMBHfQwBEIQ4dUDCA4&uact=5'.format(name)
        driver.get(link)
        
        source = driver.page_source
        soup = bs4.BeautifulSoup(source, 'html.parser')
        
        time.sleep(10)
        
        return soup.find('div',{'class':'sXLaOe'}).text
    except:
        pass



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\aparg\.wdm\drivers\chromedriver\win32\93.0.4577.63]


In [3]:
def get_coordinates(name):
    try:
        name="+".join(name.split()).lower()
        link = 'https://www.google.com/search?q={}+singapore+lat+long&rlz=1C1CHBF_enIN956IN956&sxsrf=AOaemvKxGgLC6X02Iuqqlvyp7xUwJsBFug%3A1631603674694&ei=2ktAYZ_xKY2GyAOV3IGQDA&oq=ABC+Center+Singapore+Singapore+lat+long&gs_lcp=Cgdnd3Mtd2l6EANKBAhBGABQvq9eWL6vXmDsuV5oAHACeACAAYYBiAGGAZIBAzAuMZgBAKABAqABAcABAQ&sclient=gws-wiz&ved=0ahUKEwifxsar9f3yAhUNA3IKHRVuAMIQ4dUDCA4&uact=5'.format(name)
      
        driver.get(link)
        
        source = driver.page_source
        soup = bs4.BeautifulSoup(source, 'html.parser')
        
        time.sleep(10)
        
        return soup.find('div',{'class':'Z0LcW XcVN5d'}).text.split(',')
    
    except:
        pass

In [4]:
def table(l,sctype):
    df = pd.DataFrame(zip(l),columns=['Name'])
    df['Type'] = [sctype for i in range(df.shape[0])]
    df['Address'] = df['Name'].apply(get_address)
    df['coordinates1'] = df['Name'].apply(get_coordinates) 
    return df

# Special Needs Schools and Institutions in Singapore

In [5]:
l = ['Melbourne Specialist International School',
    'Integrated International School',
     'ABC Center Singapore',
    'Dover Court International School',
     'Eden School',
     'Metta School',
    'Pathlight School Campus 1',
     'Pathlight School Campus 2 (Main)',
     'Pathlight School Campus 2 (Annex)',
     'Singapore American School',
     'St. Andrew’s Autism School',
     'Towner Gardens School',
     'The Winstedt School'
    ]

In [ ]:
df1=table(l,'Schools & Institutions')
df1

# Special Needs Preschools in Singapore

In [ ]:
l=['Modern Montessori International Katong by The Growing Academy',
  'Bright Path Preschool',
  'Dyslexia Association of Singapore',
  'Genesis School for Special Education',
  'KidzRock International Preschool',
  'Mighty Oaks',
  'Nurture Pods',
  'Wee Care']

In [ ]:
df2=table(l,'Preschools')

# Early Intervention Centres

In [ ]:
l=['The Growing Academy',
  'All Hands Together',
  'Bridge Learning',
  'The Early Intervention Centre (EIC)',
  'Kaleidoscope',
  'KidsFirst',
  'Thumbs Up Therapy Singapore'
  ]

In [ ]:
df3=table(l,'Early Intervention Centres')

# Combine All

In [ ]:
df = pd.concat([df1,df2,df3]).reset_index(drop=True)

In [ ]:
# Rearrange columns
df=df[['Type','Name', 'Address', 'coordinates1']]
df

# Fill missing Addresses

In [ ]:
df.iloc[13,2]='865 Mountbatten Road, #05-44, Singapore'
df.iloc[15,2]='1 Jurong West Central 2, #05-01 Jurong Point, Singapore'

# Fill missing Coordinates

In [ ]:
geolocator = Nominatim(user_agent="sample app")

In [ ]:
df['TempAddress']=df['Address'].apply(lambda x: " ".join(x.split()[:-1]) if x else None)
df["loc"] = df["TempAddress"].apply(geolocator.geocode)
df["coordinates2"]= df["loc"].apply(lambda loc: [loc.point.latitude,loc.point.longitude] if loc else None)

In [ ]:
df=df.drop(['loc','TempAddress'],axis=1)

In [ ]:
l=[]
for i in range(df.shape[0]):
    if(df.iloc[i,-2]!=None):
        l.append(df.iloc[i,-2])
    else:
        if(df.iloc[i,-1]!=None):
            l.append(df.iloc[i,-1])
        else:
            l.append(np.nan)
            
df['coordinates']=l

In [ ]:
# Rearrange columns
df=df[['Type','Name', 'Address', 'coordinates']]
df

In [ ]:
l1=[]
for i in df['coordinates']:
    
    try:
        l2=[]
        for j in i:
            s = float(''.join(c for c in j if c.isdigit() or c=='.'))
            l2.append(s)
        l1.append(l2)
        
    except:
        l1.append(i)

In [ ]:
df['coordinates']=l1

In [ ]:
df

In [ ]:
# manual search
df.iloc[13,-1]=[1.303940,103.901352] # Modern Montessori International
df.iloc[15,-1] = [1.339540,103.706910] # Dyslexia Association of Singapore
df.iloc[17,-1]=[1.338110,103.793400] # KidzRock International Preschool
df.iloc[24,-1] = [1.327310,103.846600] # The Early Intervention Centre (EIC)

In [ ]:
df['lat']=df['coordinates'].apply(lambda x:x[0])
df['long']=df['coordinates'].apply(lambda x:x[1])

In [ ]:
#df3.to_excel('Places.xlsx',index=False)

In [ ]:
df3.to_dict()

# Get User's Location

In [ ]:
user = '21 Lower Kent Ridge Rd, Singapore'
data = geolocator.geocode(user)

In [ ]:
data.raw

In [ ]:
lat1 = data.point.latitude
lng1 = data.point.longitude

# Get Distance

In [ ]:
from geopy.distance import geodesic

pt1 = (lat1, lng1)

l=[]
for i in range(df.shape[0]):
    lat2=df.iloc[i,-2]
    lng2=df.iloc[i,-1]
    pt2 = (lat2, lng2)
    l.append(geodesic(pt1, pt2).km)

In [ ]:
df3=df.copy()
df3['Distance']=l

In [ ]:
df3.sort_values(by=['Distance'],ascending=False).reset_index(drop=True).iloc[:5,:]